In [5]:
import graphviz
import json
import sqlite3
import sys, os, argparse
import math
import time
import collections

In [6]:
def run_sql_query(db, query):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(query)
    result_data = json.dumps(cur.fetchall())
    result_json = json.loads(result_data)
    return result_json

def gen_records(graph, nodes, edge_dict_list):
    graph.attr('node', shape='record', fontname='Courier')
    #graph.node_attr['height'] = "10"
    #graph.node_attr['width'] = "15"
    graph.node_attr['fontname'] = "Courier"
    graph.node_attr['color'] = "lightgreen"
    graph.node_attr['fillcolor'] = "lightblue"
    graph.node_attr['style'] = 'filled'
    graph.node_attr['fontsize'] = "15"
    graph.edge_attr['fontname'] = "Courier"
    graph.edge_attr['arrowsize'] = "1"
    graph.graph_attr['layout'] = "dot"
    graph.graph_attr['rankdir'] = "LR"
    #graph.graph_attr["size"] = "1500,1500"

    for node in nodes:
        graph.node(node.get("id"), 
                   node.get("txt"), 
                   fillcolor=node.get("fillcolor"),
                   rank=node.get("rank"))

    edges = []
    for edge_dict in edge_dict_list:
        graph.edge(
            edge_dict.get("src"),
            edge_dict.get("dest"),
            label=edge_dict.get("label"),
            penwidth=edge_dict.get("penwidth")
        )

def gen_node(node_id, node_txt, fillcolor, rank):
    node = {}
    node["id"] = node_id
    node["txt"] = node_txt
    node["fillcolor"] = fillcolor
    node["rank"] = rank
    return node


In [7]:
def gen_full_comparts_graph(db, graph):
    #get_compart_names_q = "SELECT DISTINCT compart_name FROM comparts"
    #compart_names_json = run_sql_query(db, get_compart_names_q)
    #get_caps_q = "select distinct cap_info.*, vm.compart_id, comparts.compart_name from cap_info inner join vm on cap_info.cap_loc_path=vm.mmap_path left join comparts on vm.compart_id=comparts.compart_id where vm.compart_id is not '-1';"

    nodes = []
    edges = []
    fillcolor = "lightblue"
    rank = "max"
    src_dest = {}

    get_default_comparts_q = "SELECT * FROM comparts WHERE is_default = 1"
    default_comparts_q_result = run_sql_query(db, get_default_comparts_q)
    default_comparts = []
    for default_compart_result in default_comparts_q_result:
        default_compart = {}
        default_compart["compart_id"] = default_compart_result[0]
        default_compart["compart_name"] = default_compart_result[1]
        default_compart["compart_start_addr"] = int(default_compart_result[3], base=16)
        default_compart["compart_end_addr"] = int(default_compart_result[4], base=16)
        default_comparts.append(default_compart)

        nodes.append(gen_node(str(default_compart["compart_id"]), 
                              str(default_compart["compart_id"])+":"+default_compart["compart_name"], 
                              "lightgreen", 
                              rank))
        
    get_sub_comparts_q = "SELECT * FROM comparts WHERE is_default = 0"
    sub_comparts_q_result = run_sql_query(db, get_sub_comparts_q)
    sub_comparts = []
    for sub_compart_result in sub_comparts_q_result:
        sub_compart = {}
        sub_compart["compart_id"] = sub_compart_result[0]
        sub_compart["compart_name"] = sub_compart_result[1]
        sub_compart["compart_start_addr"] = int(sub_compart_result[3], base=16)
        sub_compart["compart_end_addr"] = int(sub_compart_result[4], base=16)
        sub_compart["parent_id"] = sub_compart_result[6]
        sub_comparts.append(sub_compart)
            
        nodes.append(gen_node(str(sub_compart["compart_id"]), 
                              str(sub_compart["compart_id"])+":"+sub_compart["compart_name"], 
                              fillcolor, 
                              rank))

    # Need to query for RTLD addresses specification because the comparts table does not contain the right addresses for RTLD
    get_rtld_addrs_q = "SELECT start_addr, end_addr FROM vm WHERE mmap_path like '%ld-elf.so%'"
    rtld_addrs_result = run_sql_query(db, get_rtld_addrs_q)
    rtld_start_addr = int(rtld_addrs_result[0][0], base=16)
    rtld_end_addr = int(rtld_addrs_result[len(rtld_addrs_result)-1][1], base=16)
    
    get_caps_q = "SELECT * FROM cap_info"
    caps_q_result = run_sql_query(db, get_caps_q)
    caps = []
    for cap_result in caps_q_result:
        cap_dict = {}
        cap_dict["cap_loc_addr"] = int(cap_result[0], base=16)
        cap_dict["cap_addr"] = int(cap_result[2], base=16)
        cap_dict["cap_perms"] = cap_result[3]
        caps.append(cap_dict)
        
        penwidth_weight = 1

        src_comp_id = ""
        dest_comp_id = ""
        
        # First search through all the sub-compartments to find where the cap contains in
        for src_sub_compart in sub_comparts: 
            if cap_dict["cap_loc_addr"] >= src_sub_compart["compart_start_addr"] and \
               cap_dict["cap_loc_addr"] <= src_sub_compart["compart_end_addr"]:
                src_comp_id = str(src_sub_compart["compart_id"])
                src_comp_name = src_sub_compart["compart_name"]

                # Found that this cap is located in one of the sub-compartments, now look for
                # where its address is (dest). Again we search through all the sub-compartments first then look at the defaults
                for dest_sub_compart in sub_comparts:
                    if cap_dict["cap_addr"] >= dest_sub_compart["compart_start_addr"] and \
                       cap_dict["cap_addr"] <= dest_sub_compart["compart_end_addr"]: #and \
                        #dest_sub_compart["compart_id"] != src_sub_compart["compart_id"]:
                        
                        dest_comp_id = str(dest_sub_compart["compart_id"])
                        dest_comp_name = dest_sub_compart["compart_name"]
                    
                        # Found both src/dest for this cap, we can add the edge then move on to the next cap
                        if src_comp_id in src_dest:
                            src_dest[src_comp_id].append(dest_comp_id)
                        else:
                            src_dest[src_comp_id] = [dest_comp_id]

                        for props in edges:
                            if props.get("src") == src_comp_id and props.get("dest") == dest_comp_id:
                                penwidth_weight = math.log((10**(float(props.get("penwidth"))) + 1), 10)
                                edges.remove(props)
                                break
                        edges.append({"src":src_comp_id, "dest":dest_comp_id, "label":"", "penwidth":str(penwidth_weight)})
                        break #for dest_sub_compart in sub_comparts:
                # if cap_addr is not in any of the sub-compartments, try to look for it in the default compartments
                if dest_comp_id == "":
                    for dest_default_compart in default_comparts:

                            # Need to find out all the address ranges for default compartments
                            # default_comp_start_q = "SELECT start_addr FROM vm WHERE compart_id=" + str(dest_default_compart["compart_id"])
                            # default_comp_end_q = "SELECT end_addr FROM vm WHERE compart_id=" + str(dest_default_compart["compart_id"])
                            # default_comp_start_addrs = run_sql_query(db, default_comp_start_q)
                            # default_comp_end_addrs = run_sql_query(db, default_comp_end_q)

                            # for default_comp_addr_index in range(len(default_comp_start_addrs)):
                            #     default_comp_start_addr = default_comp_start_addrs[default_comp_addr_index][0]
                            #     default_comp_end_addr = default_comp_end_addrs[default_comp_addr_index][0]

                            #     if cap_dict["cap_addr"] >= int(default_comp_start_addr, base=16) and \
                            #        cap_dict["cap_addr"] <= int(default_comp_end_addr, base=16):
                        if dest_default_compart["compart_name"] == "[RTLD]":
                            dest_start_addr = rtld_start_addr
                            dest_end_addr = rtld_end_addr
                        else:
                            dest_start_addr = dest_default_compart["compart_start_addr"]
                            dest_end_addr = dest_default_compart["compart_end_addr"]
                        if cap_dict["cap_addr"] >= dest_start_addr and \
                           cap_dict["cap_addr"] <= dest_end_addr: #and \
                            #dest_default_compart["compart_id"] != src_sub_compart["compart_id"]:
                            
                            dest_comp_id = str(dest_default_compart["compart_id"])
                            dest_comp_name = dest_default_compart["compart_name"]
                        
                            # Found both src/dest for this cap, we can add the edge then move on to the next cap
                            if src_comp_id in src_dest:
                                src_dest[src_comp_id].append(dest_comp_id)
                            else:
                                src_dest[src_comp_id] = [dest_comp_id]

                            for props in edges:
                                if props.get("src") == src_comp_id and props.get("dest") == dest_comp_id:
                                    penwidth_weight = math.log((10**(float(props.get("penwidth"))) + 1), 10)
                                    edges.remove(props)
                                    break
                            edges.append({"src":src_comp_id, "dest":dest_comp_id, "label":"", "penwidth":str(penwidth_weight)})
                            break #for dest_default_compart in default_comparts:

                #break #src_sub_compart in sub_comparts

        # if cap_loc_addr is not in any of the sub-compartments, try to look for it in the default compartments
        if src_comp_id == "":
            for src_default_compart in default_comparts: 

                # Need to find out all the address ranges for default compartments
                # default_comp_start_q = "SELECT start_addr FROM vm WHERE compart_id=" + str(src_default_compart["compart_id"])
                # default_comp_end_q = "SELECT end_addr FROM vm WHERE compart_id=" + str(src_default_compart["compart_id"])
                # default_comp_start_addrs = run_sql_query(db, default_comp_start_q)
                # default_comp_end_addrs = run_sql_query(db, default_comp_end_q)

                # for default_comp_addr_index in range(len(default_comp_start_addrs)):
                #     default_comp_start_addr = default_comp_start_addrs[default_comp_addr_index][0]
                #     default_comp_end_addr = default_comp_end_addrs[default_comp_addr_index][0]

                #     if cap_dict["cap_loc_addr"] >= int(default_comp_start_addr, base=16) and \
                #        cap_dict["cap_loc_addr"] <= int(default_comp_end_addr, base=16):

                if src_default_compart["compart_name"] == "[RTLD]":
                    src_start_addr = rtld_start_addr
                    src_end_addr = rtld_end_addr
                else:
                    src_start_addr = src_default_compart["compart_start_addr"]
                    src_end_addr = src_default_compart["compart_end_addr"]
                if cap_dict["cap_loc_addr"] >= src_start_addr and \
                   cap_dict["cap_loc_addr"] <= src_end_addr:
                    src_comp_id = str(src_default_compart["compart_id"])
                    src_comp_name = src_default_compart["compart_name"]
                
                    # Found that this cap is located in one of the default compartments, now look for
                    # where its address is (dest). Again we search through all the sub-compartments first then look at the defaults
                    for dest_sub_compart in sub_comparts:
                        if cap_dict["cap_addr"] >= dest_sub_compart["compart_start_addr"] and \
                           cap_dict["cap_addr"] <= dest_sub_compart["compart_end_addr"]: #and \
                            #dest_sub_compart["compart_id"] != src_default_compart["compart_id"]:
                            
                            dest_comp_id = str(dest_sub_compart["compart_id"])
                            dest_comp_name = dest_sub_compart["compart_name"]
                        
                            # Found both src/dest for this cap, we can add the edge then move on to the next cap
                            if src_comp_id in src_dest:
                                src_dest[src_comp_id].append(dest_comp_id)
                            else:
                                src_dest[src_comp_id] = [dest_comp_id]

                            for props in edges:
                                if props.get("src") == src_comp_id and props.get("dest") == dest_comp_id:
                                    penwidth_weight = math.log((10**(float(props.get("penwidth"))) + 1), 10)
                                    edges.remove(props)
                                    break
                            edges.append({"src":src_comp_id, "dest":dest_comp_id, "label":"", "penwidth":str(penwidth_weight)})
                            break
                    # if cap_addr is not in any of the sub-compartments, try to look for it in the default compartments
                    if dest_comp_id == "":
                        for dest_default_compart in default_comparts:
                            # Need to find out all the address ranges for default compartments
                            # default_comp_start_q = "SELECT start_addr FROM vm WHERE compart_id=" + str(dest_default_compart["compart_id"])
                            # default_comp_end_q = "SELECT end_addr FROM vm WHERE compart_id=" + str(dest_default_compart["compart_id"])
                            # default_comp_start_addrs = run_sql_query(db, default_comp_start_q)
                            # default_comp_end_addrs = run_sql_query(db, default_comp_end_q)

                            # for default_comp_addr_index in range(len(default_comp_start_addrs)):
                            #     default_comp_start_addr = default_comp_start_addrs[default_comp_addr_index][0]
                            #     default_comp_end_addr = default_comp_end_addrs[default_comp_addr_index][0]

                            # if cap_dict["cap_addr"] >= int(default_comp_start_addr, base=16) and \
                            #    cap_dict["cap_addr"] <= int(default_comp_end_addr, base=16):

                            if dest_default_compart["compart_name"] == "[RTLD]":
                                dest_start_addr = rtld_start_addr
                                dest_end_addr = rtld_end_addr
                            else:
                                dest_start_addr = dest_default_compart["compart_start_addr"]
                                dest_end_addr = dest_default_compart["compart_end_addr"]
                            if cap_dict["cap_addr"] >= dest_start_addr and \
                               cap_dict["cap_addr"] <= dest_end_addr: #and \
                                   #dest_default_compart["compart_id"] != src_default_compart["compart_id"]:
                                
                                dest_comp_id = str(dest_default_compart["compart_id"])
                                dest_comp_name = dest_default_compart["compart_name"]
                        
                                # Found both src/dest for this cap, we can add the edge then move on to the next cap
                                if src_comp_id in src_dest:
                                    src_dest[src_comp_id].append(dest_comp_id)
                                else:
                                    src_dest[src_comp_id] = [dest_comp_id]

                                for props in edges:
                                    if props.get("src") == src_comp_id and props.get("dest") == dest_comp_id:
                                        penwidth_weight = math.log((10**(float(props.get("penwidth"))) + 1), 10)
                                        edges.remove(props)
                                        break
                                edges.append({"src":src_comp_id, "dest":dest_comp_id, "label":"", "penwidth":str(penwidth_weight)})
                                break
                    #break

    count_map = {}
    for idx,values in src_dest.items():
        for val in values:
            if str(idx)+":"+str(val) in count_map:
                count_map[str(idx)+":"+str(val)] += 1
            else :
                count_map[str(idx)+":"+str(val)] = 1

    print(count_map)
    gen_records(graph, nodes, edges)

In [8]:
dbpath="../../chericat_dbs/"
dbname="dlopen.db"
start = time.perf_counter()
digraph = graphviz.Digraph('G', filename=dbname+'.comparts_no_self_refs_view_graph.gv')
gen_full_comparts_graph(dbpath+dbname, digraph)
end = time.perf_counter()
print("Full compartments graph generation time taken: " + str(end-start) + "s")
        
digraph.render(directory='graph-output', view=True)

{'3:3': 70, '3:0': 1, '3:8': 1, '3:11': 1, '3:9': 1, '3:10': 1, '4:3': 2, '4:0': 1, '4:4': 2, '4:16': 1, '4:14': 1, '5:30': 1, '5:3': 8, '5:0': 1, '5:5': 10, '5:21': 1, '6:3': 4, '6:0': 1, '6:6': 6, '7:3': 6, '7:0': 1, '7:9': 1, '7:11': 1, '7:10': 1, '7:12': 1, '7:13': 1, '8:3': 5, '8:0': 1, '8:8': 3, '8:23': 1, '8:20': 1, '9:3': 5, '9:0': 1, '9:9': 5, '10:3': 3, '10:0': 1, '10:10': 5, '11:3': 1, '11:22': 1, '11:6': 1, '11:15': 1, '12:3': 4, '12:0': 1, '12:12': 5, '12:22': 1, '12:6': 1, '12:15': 1, '13:3': 4, '13:0': 1, '13:13': 7, '13:22': 1, '13:15': 1, '13:5': 1, '14:3': 7, '14:0': 1, '14:14': 4, '15:0': 1, '15:15': 1, '16:3': 5, '16:0': 1, '16:16': 7, '17:3': 2, '17:0': 1, '18:3': 1, '18:0': 1, '18:18': 1, '19:3': 2, '19:0': 1, '19:19': 2, '19:24': 1, '20:30': 1, '20:3': 3, '20:0': 1, '20:20': 2, '20:7': 1, '20:27': 1, '20:17': 1, '20:28': 1, '20:18': 1, '20:19': 1, '21:0': 1, '21:21': 2, '22:0': 1, '22:22': 2, '23:3': 11, '23:0': 1, '23:23': 1, '24:3': 4, '24:0': 1, '24:24': 7, '2

'graph-output/server_0408.db.comparts_no_self_refs_view_graph.gv.pdf'